# Data & Training

In [1]:
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm

import torch

from Training import evaluate, train
from Data import AudioDataset, retrieve_data

In [5]:
directory = ""#"latents\\" #"reconstruction_test_latents\\" #mtg-jamendo
num_per = 1000
count = 1
for start in tqdm(range(5000, 16150, num_per)):
    mtg_dataset = retrieve_data("E:\SongsDataset\\mtg-jamendo\\", directory, start=start, count=num_per, sample_length=256)
    torch.save(mtg_dataset, f"E:\\SongsDataset\\dataset{count}.pt")

    count += 1

100%|██████████| 12/12 [11:05<00:00, 55.49s/it]


In [7]:
directory = ""#"latents\\" #"reconstruction_test_latents\\" #mtg-jamendo
num_per = 1000
count = 18
for start in tqdm(range(0, 3973, num_per)):
    spotify_dataset = retrieve_data("E:\SongsDataset\\latents\\", directory, start=start, count=num_per, sample_length=256)
    torch.save(spotify_dataset, f"E:\\SongsDataset\\dataset{count}.pt")

    count += 1

100%|██████████| 4/4 [02:48<00:00, 42.16s/it]


In [9]:
full_dataset = torch.load(f"E:\\SongsDataset\\dataset1.pt")
for start in tqdm(range(2, 21)):
    new_data = torch.load(f"E:\\SongsDataset\\dataset{start}.pt")
    full_dataset = torch.cat((full_dataset, new_data))

print(full_dataset.shape)

100%|██████████| 19/19 [01:36<00:00,  5.09s/it]

torch.Size([151741, 256, 64])


In [10]:
torch.save(full_dataset, "E:\\SongsDataset\\full_dataset.pt")

In [2]:
#full_dataset = torch.load("E:\\SongsDataset\\full_dataset.pt")
full_dataset = torch.load("E:\\SongsDataset\\dataset1.pt")

In [3]:
# ==== Model & Optimizer ====
num_heads = 8
num_layers = 8
d_model = 512
latent_space = 64
dim_feedforward = 512

batch_size = 24

device = "cuda"

In [4]:
from torch.utils.data import random_split

num_samples, seq_length, embed_dim = full_dataset.shape

train_len = int(len(full_dataset) * 0.9)
train_set, test_set = random_split(full_dataset, [train_len, len(full_dataset) - train_len])

train_dataset = AudioDataset(train_set)
test_dataset = AudioDataset(test_set)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [5]:
from Analyzer.Webscraper.AudioTransformerCNNReconstruction import AudioTransformerCNNReconstruction

model = AudioTransformerCNNReconstruction(d_model=d_model, num_heads=num_heads, transformer_layers=num_layers, dim_feedforward=dim_feedforward, latent_space=latent_space, length=256, dropout=0.1001)

In [6]:
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-5)

In [7]:
train(model, train_dataloader, test_dataloader, optimizer, num_epochs=4, device=device)

100%|██████████| 44/44 [00:03<00:00, 13.38it/s]


Epoch [1/100], Training Loss: 0.9629 	 Validation Loss: 0.9629


100%|██████████| 44/44 [00:03<00:00, 12.71it/s]


Epoch [2/100], Training Loss: 0.8633 	 Validation Loss: 0.8633


100%|██████████| 44/44 [00:03<00:00, 12.94it/s]


...

Epoch [9/100], Training Loss: 0.7239 	 Validation Loss: 0.7239


100%|██████████| 44/44 [00:03<00:00, 13.77it/s]


Epoch [10/100], Training Loss: 0.7184 	 Validation Loss: 0.7184


 27%|██▋       | 104/390 [00:26<01:13,  3.89it/s]


KeyboardInterrupt: 

In [4]:
from torch.utils.data import random_split

reconstruction_examples = retrieve_data("E:\SongsDataset\\",  "reconstruction_test_latents\\", sample_length=256)

# ==== Data Preparation ====
song_set, _ = random_split(reconstruction_examples, [len(reconstruction_examples), 0])
song_dataset = AudioDataset(song_set)
song_dataloader = DataLoader(song_dataset, batch_size=batch_size, shuffle=True)

# Output Stuff

In [1]:
model = torch.load("AudioTransformerCNNReconstruction-LatentSpace64-Heads8-TrasformerLayers6-DModel256-Dropout0.1\\-Epoch-6.pt", weights_only=False)
evaluate(model, song_dataloader)

NameError: name 'torch' is not defined

In [7]:
# Output Stuff
model = torch.load(
    "AudioTransformerCNNReconstruction-LatentSpace64-Heads8-TrasformerLayers8-DModel256-Dropout0.1\\-Epoch-13.pt",
    weights_only=False)
evaluate(model, song_dataloader)

100%|██████████| 18/18 [00:00<00:00, 23.44it/s]


Avg Reconstructive Loss: 0.5488527417
Avg Contrastive Loss: 0.1242191568
Avg Total Loss: 0.6731


0.6730718844466739

In [ ]:
from Loss import combined_loss

device = "cuda"

model.eval()  # Set model to evaluation mode
model.to(device)
total_loss = 0.0
num_batches = 0

new_song = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in tqdm(song_dataloader):
        batch = batch.to(device)

        # Forward pass
        reconstructed = model(batch)

        new_song.extend(reconstructed.to("cpu"))

        loss = combined_loss(reconstructed, batch)

        total_loss += loss.item()
        num_batches += 1

In [ ]:
from Loss import combined_loss

device = "cuda"

model.eval()  # Set model to evaluation mode
model.to(device)
total_loss = 0.0
num_batches = 0

latent_space = []

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in tqdm(song_dataloader):
        batch = batch.to(device)

        # Forward pass
        latent = model.to_latent(batch)

        latent_space.extend(latent.to("cpu"))

        num_batches += 1

In [ ]:
print(len(latent_space))
print(len(latent_space[0]))
print(len(latent_space[0][0]))

In [ ]:
l = np.array(np.stack(new_song))
np.save("reconstructed_song-256-Smaller-96.npy", l.transpose())

In [ ]:
import IPython
import numpy as np
from music2latent import EncoderDecoder

encdec = EncoderDecoder()

In [ ]:
compressed_song = np.load("reconstructed_song-256-Bigger.npy")
wv_rec = encdec.decode(compressed_song)
IPython.display.display(IPython.display.Audio(wv_rec, rate=44100))

In [ ]:
compressed_song = np.load("reconstructed_song-256-Smaller-13.npy")
wv_rec = encdec.decode(compressed_song)
IPython.display.display(IPython.display.Audio(wv_rec, rate=44100))

In [ ]:
compressed_song = np.load("reconstructed_song-256-Smaller-96.npy")
wv_rec = encdec.decode(compressed_song)
IPython.display.display(IPython.display.Audio(wv_rec, rate=44100))

NameError: name 'nvcc' is not defined